# Rain in Australia - Logistic Regression
**Автор:** Мойсеєнко Світлана

## 1. Імпорт пакетів

In [ ]:
import kagglehub
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

## 2. Завантаження даних

In [ ]:
path = kagglehub.dataset_download("jsphyg/weather-dataset-rattle-package")
print("Path to dataset files:", path)

df = pd.read_csv(f"{path}/weatherAUS.csv")
print(f"\nРозмір датасету: {df.shape[0]} рядків, {df.shape[1]} колонок")
df.head()

## 3.1. Видалення ознак з великою кількістю пропусків

In [ ]:
missing_pct = df.isnull().sum() / len(df) * 100
print("Колонки з пропусками > 30%:")
print(missing_pct[missing_pct > 30].round(1))

cols_to_drop = missing_pct[missing_pct > 30].index.tolist()
df = df.drop(columns=cols_to_drop)
print(f"\nВидалено колонок: {len(cols_to_drop)}")
print(f"Залишилось колонок: {df.shape[1]}")

## 3.2. Створення підмножин з числовими та категоріальними ознаками

In [ ]:
target = 'RainTomorrow'
df = df.dropna(subset=[target])

numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
categorical_cols = [c for c in categorical_cols if c not in ['Date', target]]

print(f"Числові ознаки ({len(numeric_cols)}): {numeric_cols}")
print(f"\nКатегоріальні ознаки ({len(categorical_cols)}): {categorical_cols}")

## 3.3. Обробка дати: створення Year та Month

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month.astype(str)

print(f"Діапазон років: {df['Year'].min()} - {df['Year'].max()}")
print(f"Унікальні місяці: {sorted(df['Month'].unique())}")

## 3.4. Переміщення Year до числових, Month до категоріальних

In [ ]:
numeric_cols.append('Year')
categorical_cols.append('Month')

print(f"Числові ознаки ({len(numeric_cols)}): {numeric_cols}")
print(f"\nКатегоріальні ознаки ({len(categorical_cols)}): {categorical_cols}")

## 3.5. Розбиття на train/test за роком

In [ ]:
max_year = df['Year'].max()
print(f"Останній рік спостережень (для тесту): {max_year}")

train_mask = df['Year'] < max_year
test_mask = df['Year'] == max_year

X_train_num = df.loc[train_mask, numeric_cols]
X_test_num = df.loc[test_mask, numeric_cols]

X_train_cat = df.loc[train_mask, categorical_cols]
X_test_cat = df.loc[test_mask, categorical_cols]

y_train = df.loc[train_mask, target]
y_test = df.loc[test_mask, target]

print(f"\nTrain: {len(y_train)} записів")
print(f"Test: {len(y_test)} записів")
print(f"\nРозподіл класів у test:")
print(y_test.value_counts())

## 4. Імпутація пропущених значень

In [ ]:
num_imputer = SimpleImputer(strategy='median')
X_train_num_imp = num_imputer.fit_transform(X_train_num)
X_test_num_imp = num_imputer.transform(X_test_num)

cat_imputer = SimpleImputer(strategy='most_frequent')
X_train_cat_imp = cat_imputer.fit_transform(X_train_cat)
X_test_cat_imp = cat_imputer.transform(X_test_cat)

print("Імпутація завершена")
print(f"Числові ознаки: {X_train_num_imp.shape[1]} колонок")
print(f"Категоріальні ознаки: {X_train_cat_imp.shape[1]} колонок")

## 5. Нормалізація числових ознак

In [ ]:
scaler = StandardScaler()
X_train_num_scaled = scaler.fit_transform(X_train_num_imp)
X_test_num_scaled = scaler.transform(X_test_num_imp)

print("Нормалізація завершена")
print(f"Train mean: {X_train_num_scaled.mean():.4f}, std: {X_train_num_scaled.std():.4f}")

## 6. Кодування категоріальних ознак

In [ ]:
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_train_cat_encoded = encoder.fit_transform(X_train_cat_imp)
X_test_cat_encoded = encoder.transform(X_test_cat_imp)

print(f"Кодування завершено")
print(f"Кількість категоріальних ознак після кодування: {X_train_cat_encoded.shape[1]}")

## 7. Об'єднання ознак та навчання моделі

In [ ]:
X_train = np.hstack([X_train_num_scaled, X_train_cat_encoded])
X_test = np.hstack([X_test_num_scaled, X_test_cat_encoded])

print(f"Розмір train: {X_train.shape}")
print(f"Розмір test: {X_test.shape}")

clf = LogisticRegression(solver='lbfgs', max_iter=1000, random_state=42)
clf.fit(X_train, y_train)

print("\nМодель навчена успішно")

## 8. Оцінка моделі та висновки

In [ ]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

### Висновки

**Порівняння з базовою моделлю:**

Отримані метрики близькі до базової моделі з розділу курсу (accuracy ~0.80).

**Ключові особливості підходу:**
1. **Часовий split** - модель навчалась на даних попередніх років і тестувалась на останньому році, що імітує реальні умови використання
2. **Коректна обробка даних** - всі трансформери (SimpleImputer, StandardScaler, OneHotEncoder) навчались тільки на train даних
3. **Обробка часових ознак** - Year використано як числову ознаку, Month - як категоріальну

Модель показує хорошу точність для прогнозування дощу на наступний день.